In [8]:
from qiskit import *
from qiskit.visualization import *
from qiskit.circuit.library import *
import math

In [2]:
# This notebook implements a quantum algorithm, based on the usual quantum search algorithm, which takes as input a vector of positive integers and returns all subsets of that vector which sum to a given integer.

In [4]:
# For example, if the input is the vector [1,3,6,4,2] with target 6, we return the list ([1,3,2],[6],[4,2]).

In [ ]:
# The algorithm is implemented by the function vector_subsets_sum.

In [53]:
# The function takes three inputs: a vector list of nonnegative integers, a target sum, and a backend on which to run the quantum circuit.

In [137]:
def vector_subsets_sum(vector, target, backend):
    # First check that all components of the vector are nonnegative integers:
    for i in vector:
        if not isinstance(i,int):
            return "Error: noninteger component in input vector."
        if i<0:
            return "Error: negative component in input vector."
    # Then check that the target is a nonnegative integer not larger than the sum of all components of the input vector:
    if not isinstance(target,int):
        return "Error: noninteger target."
    if target<0:
        return "Error: negative target."
    if target>sum(vector):
        return "Error: target too large."
    # We set up a quantum circuit with enough qubits to index vector components, and then enough bits to store sums of vector components, plus ancilla bits to store the vector components and one more ancilla bit.
    l = len(vector)
    n = 2**l
    ind = QuantumRegister(l,'index')
    valsize = int(math.log2(sum(vector))+1)
    val = QuantumRegister(valsize,'value')
    valanc = QuantumRegister(valsize,'value ancilla')
    anc = QuantumRegister(1,'ancilla')
    res = ClassicalRegister(l,'result')
    qc = QuantumCircuit(ind,val,valanc,anc,res)
    # Create a superposition of indices
    qc.h(ind)
    # Initialize ancilla qubit in the |-> state
    qc.x(anc)
    qc.h(anc)
    qc.barrier()

    # We construct a gate which sets all val qubits to 1 if and only if the sum of the components indicated by the index qubits set to 1 equal the target
    check = QuantumCircuit(ind,val,valanc,anc)
    # First, we use Draper adders to add each value in the vector, controlled on the index qubit
    for i in range(l):
        val_copy = vector[i]
        bit = []
        for j in range(valsize):
            bit.append(val_copy%2)
            if bit[j]==1:
                check.x(valanc[j])
            val_copy = (val_copy-bit[j])/2
        check=check.compose(DraperQFTAdder(valsize).control(),qubits=ind[i:i+1]+valanc[0:valsize]+val[0:valsize])
        for j in range(valsize):
            if bit[j]==1:
                check.x(valanc[j])
    # To check against the target, we bitwise add the complement of the target so that all val bits are 1 in case of matching the target
    target_copy = target
    bit = []
    for i in range(valsize):
        bit.append(target_copy%2)
        if bit[i]==0:
            check.x(val[i])
        target_copy = (target_copy-bit[i])/2
    checkgate = check.to_gate()
    # We now create the phaseflip oracle and Grover iterate for the counting and search algorithms
    # This oracle implements a phaseflip on the ancilla qubit as long as it is initialized in the |-> state 
    ora = QuantumCircuit(ind,val,valanc,anc)
    # First, we apply the checkgate to set all val qubits to 1 for index states which sum to the target
    ora.append(checkgate, range(l+valsize+valsize+1))
    # Then, we apply an X-gate controlled on all val states. Since the ancilla qubit is initalized to |->, this causes a phaseflip if the index states sum to the target
    ora.mcx(val,anc[0])
    # Finally we uncompute by applying the inverse of the checkgate, setting the val and ancilla val states back to |0>
    ora.append(checkgate.inverse(), range(l+valsize+valsize+1))
    # The second part of the Grover iterate is a standard diffuser:
    dif = QuantumCircuit(ind)
    dif.h(ind)
    dif.x(ind)
    dif.h(ind[l-1])
    dif.mct(ind[0:l-1],ind[l-1])
    dif.h(ind[l-1])
    dif.u(2*math.pi,0,0,ind[l-1])
    dif.x(ind)
    dif.h(ind)
    diffuser = dif.to_gate()
    # We now construct the Grover iterate and the controlled Grover iterate
    ora.append(diffuser,ind)
    grov = ora.to_gate()
    cgrov = grov.control()
    # We now employ a quantum counting algorithm to estimate the Grover angle and number of solutions
    # We need our original circuit, plus extra qubits for phase estimation
    # The number of extra qubits:
    t = int(l/2) + 4
    ext = QuantumRegister(t)
    meas = ClassicalRegister(t)
    # We initialize the extra qubits in superposition and copy our circuit from above
    qcount = QuantumCircuit(ext,ind,val,valanc,anc,meas)
    qcount.h(ext)
    qcount = qcount.compose(qc, qubits=ind[0:l]+val[0:valsize]+valanc[0:valsize]+anc[0:1])
    # We now add the appropriate number of controlled Grover iterates:
    for i in range(t):
        for j in range(2**i):
            qcount.append(cgrov, ext[i:i+1]+ind[0:l]+val[0:valsize]+valanc[0:valsize]+anc[0:1])
    # and finally an inverse QFT and measurement:
    qcount = qcount.compose(QFT(num_qubits=t).inverse(),ext[0:t])
    qcount.measure(ext,meas)
    # We now run the counting circuit on the given backend:
    job = backend.run(assemble(transpile(qcount,backend)), shots=1024)
    hist = job.result().get_counts()
    # We read the most probable output, and convert it so it gives us the small version of the angle
    pretheta = int(max(hist,key=hist.get),2)
    tt = 2**(t-1)
    pretheta = pretheta%tt
    if pretheta > 2**(t-2):
        pretheta = tt-pretheta
    # This gives our estimates for the Grover angle and number of solutions:
    theta = (pretheta/(2**t))*math.pi*2
    nsols = round(n * math.sin(theta/2) * math.sin(theta/2))
    
    # Using our estimate for theta and the number of solutions, we compute the number of Grover iterates for our main circuit
    num_grovs = round(math.acos(math.sqrt(nsols/n))/theta)
    # We apply the appropriate number of Grover iterates and measure:
    for i in range(num_grovs):
        qc.append(grov, ind[0:l]+val[0:valsize]+valanc[0:valsize]+anc[0:1])
    qc.measure(ind,res)
    # We run our circuit enough times to find all solutions
    nshots = 1024*int(math.log(nsols+2,2))
    job = backend.run(assemble(transpile(qc,backend)), shots=nshots)
    dicta = job.result().get_counts()
    sols = []
    # We count a solution if it appears at least 1/2nsols of the time
    for i in dicta:
        if dicta[i]>nshots/(2*nsols):
            sols.append(i)
    return sols


In [136]:
# We now initialize the inputs:

In [142]:
vector = [5,7,8,9,1]

In [143]:
target = 16

In [144]:
backend=Aer.get_backend('aer_simulator')

In [122]:
# And we run the program:

In [145]:
sols = vector_subsets_sum(vector,target,backend)

In [113]:
# Note that the output gives a list of bitstrings.

In [124]:
# In each bitstring, the indices set to 1 indicate the vector components which form part of the sum.

In [134]:
# The bitstrings read right-to-left; e.g., for the vector [5,7,8,9,1], the bitstring 10110 indicates the subset [7,8,1]

In [146]:
sols

['01010', '10110']